In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

RUNPOD_API_KEY = os.getenv("RUNPOD_API_KEY")

In [34]:
import requests

RUNPOD_ENDPOINT_ID = 'sw0tgzmaqgxyvp'

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {RUNPOD_API_KEY}'
}

quality_tags = '(masterpiece, realistic, best quality, ultra-detailed, high resolution, intricate design, 8K, photorealistic, professional lighting)'
prompt = 'A futuristic robot girl with glowing neon circuits, sleek metallic body, and expressive cybernetic eyes, wearing a high-tech bodysuit. Standing in a sci-fi cityscape with neon lights reflecting off wet streets'

data = {
  'input': {
  "prompt": prompt + quality_tags,
  "negative_prompt": "blurry, low quality, pixelated, distorted, deformed hands, extra limbs, asymmetrical features, text, watermark",
  "width": 512,
  "height": 768,
#   "samples": 1,
#   "num_inference_steps": 50,
#   "guidance_scale": 7.5,
#   "seed": null,
#   "safety_checker": "yes",
#   "enhance_prompt": "yes",
#   "multi_lingual": "no",
#   "panorama": "no",
#   "self_attention": "yes",
#   "upscale": "no",
#   "embeddings_model": "default",
#   "webhook": null,
#   "track_id": "robotgirl_001"
}
}

# response = requests.post('https://api.runpod.ai/v2/sw0tgzmaqgxyvp/run', headers=headers, json=data)

In [ ]:
response.status_code

In [ ]:
response.json().get('id')

In [ ]:
import time

endpoint_id = 'sw0tgzmaqgxyvp'

start_time = time.time()
#response = requests.post(f'https://api.runpod.ai/v2/{endpoint_id}/run', headers=headers, json=data)

status_headers = {
    'Authorization': f'Bearer {RUNPOD_API_KEY}'
}

job_id = response.json().get('id')

while True:
    response_img = requests.get(f'https://api.runpod.ai/v2/{endpoint_id}/status/{job_id}', headers=status_headers)
    status = response_img.json().get('status')
    if status == "COMPLETED":
        break
    time.sleep(1)

elapsed_time = time.time() - start_time

print(f"Completed in {elapsed_time:.2f} seconds.")
image = response_img.json().get('output')[0].get('image')

from utils import display_image

#display_image(image, max_size=1000)


In [ ]:
response_img.json()

In [ ]:
import time
import threading
from utils import display_image

def process_request(thread_id, data):

    start_time = time.time()

    response = requests.post('https://api.runpod.ai/v2/sw0tgzmaqgxyvp/run', headers=headers, json=data)
    job_id = response.json().get('id')

    status_headers = {'Authorization': f'Bearer {RUNPOD_API_KEY}'}

    while True:
        response_img = requests.get(f'https://api.runpod.ai/v2/sw0tgzmaqgxyvp/status/{job_id}', headers=status_headers)
        status = response_img.json().get('status')

        if status == "COMPLETED":
            break
        time.sleep(1)

    image = response_img.json().get('output')[0].get('image')
    elapsed_time = time.time() - start_time
    print(f"Thread {thread_id} completed in {elapsed_time:.2f} seconds.")

    display_image(image, max_size=200)

threads = []

for i in range(6):
    thread = threading.Thread(target=process_request, args=(i, data))
    threads.append(thread)
    thread.start()

# Wait for all threads to complete
for thread in threads:
    thread.join()

print("All requests completed.")